**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer:__  
The function ```act``` determines the next action to do, based on the current state.   
Without a randomness parameter, the choice returned is only based on the previous knowledge. For the training phase, it is important to explore new possibilities, the parameter ```epsilon``` the exploration/exploitation trade-of. It sets the probability for the agent to explore new actions or to exploit what it already knows.  
If $\epsilon = 0$, the function never explores, and on the contrary if $\epsilon = 1$, all the action are random. This parameter is therefore essential.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
# Changed some lines, there were some errors
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #################################################
        # There is an error in the original code
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        #################################################

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        ###############################################################
        # Same error here
        # self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        ###############################################################
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
#epochs_train=?? # set small when debugging
#epochs_test=?? # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer:__  
The array ```position``` contains:
- the former position of the rat, set to 1, 
- the other cells to which the rat can go, set to 0,
- and the borders set to -1.

The array ```board``` indicates the content of each cell of the board. Each cell can have 3 different states:
- 0 if the cell contains nothing
- 0.5 if the cell contains a cheese
- -1 if the cell is poisonous.   

At each action, the content of the cell to which the mouse move is returned as the reward and its content is set to 0.



At each step, the returned ```state``` variable contains the visible part of the ```board``` and ```position``` arrays.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0, self.n_action)
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Test {}: Win/lose count {}/{}. Average score ({})"
              .format(e, win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
size = 13
T = 200
temperature = 0.3
epochs_train = 51
epochs_test = 101

# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Test 0: Win/lose count 6.0/13.0. Average score (-7.0)
Test 1: Win/lose count 9.0/13.0. Average score (-5.5)
Test 2: Win/lose count 8.0/16.0. Average score (-6.333333333333333)
Test 3: Win/lose count 5.5/12.0. Average score (-6.375)
Test 4: Win/lose count 5.0/6.0. Average score (-5.3)
Test 5: Win/lose count 10.0/12.0. Average score (-4.75)
Test 6: Win/lose count 11.5/12.0. Average score (-4.142857142857143)
Test 7: Win/lose count 12.0/19.0. Average score (-4.5)
Test 8: Win/lose count 10.5/6.0. Average score (-3.5)
Test 9: Win/lose count 16.0/9.0. Average score (-2.45)
Test 10: Win/lose count 8.5/11.0. Average score (-2.4545454545454546)
Test 11: Win/lose count 9.0/11.0. Average score (-2.4166666666666665)
Test 12: Win/lose count 13.0/12.0. Average score (-2.1538461538461537)
Test 13: Win/lose count 10.5/23.0. Average score (-2.892857142857143)
Test 14: Win/lose count 8.5/7.0. Average score (-2.6)
Test 15: Win/lose count 5.5/10.0. Average score (-2.71875)
Test 16: Win/lose count 9.5/17.0

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer:__  
1) We have:  
$Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>$  
Therefore:  
$ Q^\pi(s,a) = E_{p^{\pi}}[r(s_0, a_0) + \sum_{t\geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $  
$ Q^\pi(s,a) = E_{p^{\pi}}[r(s_0, a_0)|s_{0}=s,a_{0}=a] + E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $  
$ Q^\pi(s,a) = r(s, a) + E_{p^{\pi}}[\gamma \sum_{t\geq 0}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] $  
$ Q^\pi(s,a) = r(s, a) + E_{p^{\pi}}[\gamma E_{p^{\pi}}[\sum_{t\geq 0}\gamma^{t}r(s_{t+1},a_{t+1})|s_1 = s', a_1 = a']|s_{0}=s,a_{0}=a] $  
$ Q^\pi(s,a) = r(s, a) + E_{(s', a')\sim p^{\pi}(.|s, a)}[\gamma E_{p^{\pi}}[\sum_{t\geq 0}\gamma^{t}r(s_{t+1},a_{t+1})|s_1 = s', a_1 = a']] $

Besides: $  Q^\pi(s', a') = E_{p^{\pi}}[\sum_{t\geq 0}\gamma^{t}r(s_{t+1},a_{t+1})|s_1=s', a_1=a'] $   
Therefore:  
$ Q^\pi(s,a) = r(s, a) + E_{p^{\pi}}[\gamma Q^\pi(s', a')|s_{0}=s,a_{0}=a] $
$ Q^\pi(s,a) = r(s, a) + E_{(s', a')\sim p^{\pi}(.|s, a)}[\gamma Q^\pi(s', a')] $  
And finally:  
$$ Q^\pi(s,a) = E_{(s', a')\sim p^{\pi}(.|s, a)}[r(s, a) + \gamma Q^\pi(s', a')] $$


2) We assume that the optimal policy $\pi^*$ exists:  
$ Q^*(s,a)=Q^{\pi^*}(s,a) $  
$ Q^*(s,a)=\max_{\pi}Q^\pi(s,a) $  
$ Q^{*}(s,a) = \max_{\pi} [E_{(s', a')\sim p^{\pi}(.|s, a)}[r(s, a) + \gamma Q^{\pi}(s', a')] ] $  
$ Q^{*}(s,a) = \max_{\pi} [r(s, a) + \gamma E_{(s', a')\sim p^{\pi}(.|s, a)}[Q^{\pi}(s', a')] ] $  
$ Q^{*}(s,a) = r(s, a) + \gamma \max_{\pi} [E_{(s', a')\sim p^{\pi}(.|s, a)}[Q^{\pi}(s', a')] ] $  
$ Q^{*}(s,a) = r(s, a) + \gamma \max_{\pi} [ \sum_{(s', a')}{p^{\pi}(s', a'|s, a)Q^{\pi}(s', a')} ] $  
$ Q^{*}(s,a) = r(s, a) + \gamma \max_{\pi} [ \sum_{(s', a')}{p^{\pi}(s_{t+1} = s', a_{t+1} = a'| s_t = s, a_t = a)Q^{\pi}(s', a')} ] $  
$ Q^{*}(s,a) = r(s, a) + \gamma \max_{\pi} [ \sum_{(s', a')}{p^{\pi}(s_{t+1} = s'| s_t = s, a_t = a)\pi(a_{t+1} = a'|s_{t+1} = s)Q^{\pi}(s', a')} ] $  
$ Q^{*}(s,a) = r(s, a) + \gamma \sum_{s'}[p^{\pi^*}(s_{t+1} = s'| s_t = s, a_t = a)\max_\pi[{\sum_{a'}{\pi(a_{t+1} = a'|s_{t+1} = s')Q^{\pi}(s', a')}] }]$  
Besides, $\pi$ is a indicator function for the policy, it equals $1$ for the action of the policy given $s'$, and $0$ for the others, therefore:  
$\max_\pi[{\sum_{a'}{\pi(a_{t+1} = a'|s_{t+1} = s')Q^{\pi}(s', a')}] } = \max_a'\max_{\pi}Q^{\pi}(s', a') = \max_a'Q^{*}(s', a') $  
Therefore:  
$ Q^{*}(s,a) = r(s, a) + \gamma \sum_{s'}[p^{\pi^*}(s_{t+1} = s'| s_t = s, a_t = a)\max_a'Q^{*}(s', a')]$  
$ Q^{*}(s,a) = r(s, a) + \gamma E_{s' \sim p^{\pi^*}(\cdot| s, a)}[\max_a'Q^{*}(s', a')]$  
$$ Q^{*}(s,a) = E_{s' \sim p^{\pi^*}(\cdot| s, a)}[r(s, a) + \gamma \max_a'Q^{*}(s', a')]$$  


3) The loss function:  
$$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$
is derived from the equation in 2) by differencing the two terms of the equality, and remarking that for $Z$ a random variable, $E[Z]^2\leq E[Z^2]$. Therefore by trying to find a $\theta$ which nullify the loss function, we try to nullify the equation from 2).  
This loss function is actually suitable for our problem.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # We store the move at the end of the list and pop the first element if we have reached the max_memory
        self.memory.append(m)
        while len(self.memory)>self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        # We draw a random index for the current length of the memory
        index = np.random.randint(0, len(self.memory))
        res = self.memory[index]
        return res

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        score = 0
        loss = 0
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss += agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
from keras.utils import to_categorical

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        
        for i in range(self.batch_size):
            ######## FILL IN
            # We draw a realisation from the pool in order to learn from it
            s_batch, n_s_batch, a_batch, r_batch, game_over_batch = self.memory.random_access()
            #We add the drawn input state to the training batch
            input_states[i, :, :, :] = s_batch
            # We add the reward to the element corresponding to the action chosen in the memory
            reward_batch = self.model.predict(s_batch.reshape(1, 5, 5, self.n_state))[0,:]
            reward_batch[a_batch] = r_batch
            
            # In the algorithm, the distinction is done on the state from wich we learn and not the current state of the learning agent
            if game_over_batch: #game_over_: 
                ######## FILL IN
                target_q[i] = reward_batch
            else:
                ######## FILL IN
                # The potential rewards from the new state
                # We assume that the first element of the state contains the visible board
                # We add this new potential reward to the of given by the former action
                prediction = self.model.predict(n_s_batch.reshape(1, 5, 5, self.n_state))
                reward_batch[a_batch] += self.discount*max(prediction[0,:])
                target_q[i] = reward_batch
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

        
from keras.layers import Dropout, Flatten            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        # We create a FCNN
        model = Sequential()
        model.add(Dense(32, input_shape = (5, 5, self.n_state)))
        model.add(Flatten())
        model.add(Dense(32, activation ='relu'))
        model.add(Dense(4, activation ='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        print(self.model.summary())
    
    def learned_act(self, s):
        # The learned action is the argmax
        scores = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        action = np.argmax(scores)
        return action

In [12]:
size = 13
T = 200
temperature = 0.3
epochs_train = 51
epochs_test = 101

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train,prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5, 5, 32)          96        
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                25632     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 132       
Total params: 25,860
Trainable params: 25,860
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/051 | Loss 2.8823 | Win/lose count 6.5/10.0 (-3.5)
Epoch 001/051 | Loss 1.5394 | Win/lose count 5.0/1.0 (4.0)
Epoch 002/051 | Loss 2.4490 | Win/lose count 3.5/2.0 (1.5)
Epoch 003/051 | Loss 1.8972 | Win/lose count 9.5/6.0 (3.5)
Epoch 004/051 | Loss 2.3393

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, (2,2), input_shape = (5, 5, self.n_state)))
        model.add(Conv2D(32, (2,2)))
        model.add(Flatten())
        model.add(Dense(32, activation ='relu'))
        model.add(Dense(4, activation = 'linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        print(self.model.summary())
        
    def learned_act(self, s):
        scores = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        action = np.argmax(scores)
        return action

In [14]:
size = 13
T = 200
temperature = 0.3
epochs_train = 51
epochs_test = 101

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 32)          4128      
_________________________________________________________________
flatten_2 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                9248      
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 132       
Total params: 13,796
Trainable params: 13,796
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/051 | Loss 1.6114 | Win/lose count 1.5/1.0 (0.5)
Epoch 001/051 | Loss 2.5026 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/051 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
size = 13
T = 200
temperature = 0.3
epochs_train = 51
epochs_test = 101

env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 32)          4128      
_________________________________________________________________
flatten_3 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                9248      
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 132       
Total params: 13,796
Trainable params: 13,796
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
d

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

The main issue that we observe is that the rat tend not to explore the environment and often get stuck between two positions. mainly happens when the rat do not see anymore reward in its surrounding. With this strategy, the score do not decrease but it do not increase anymore as well.  

For a temperature of $0.3$, we have a score of 5.9 for the CNN and 4.3 for th FCN. Let's test different temperature.

|Temperature|CNN Score|FCN Score|
|-----------|---------|---------|
|0.1        |1.7      |1.6      |
|0.3        |5.5      |4.6      |
|0.5        |11.1     |8.9      |
|0.7        |21.4     |15.8     |
|0.9        |39.7     |19.0     |

Both the scores increase with the temperature because there are more rewards, but overall, the CNN agent performs the best.   
It seems that the more the temperature rises, the more the gap between the CNN an the FCN is wide. The CNN adapts better to the increasing temperature.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix=''):
    # pass
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        loss = 0
        agent.set_epsilon(agent.epsilon*0.9)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss += agent.reinforce(prev_state, state,  action, reward, game_over)

        # Update stats
        score += win-lose

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        print("Epoch {:03d}/{:03d} | Epsilon {:.4f} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, agent.epsilon, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'exploremodel.h5',name_model=prefix+'exploremodel.json')


class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        # pass
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        # changed########################################
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        #################################################

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        ###############################################################
        #changed
        # self.position[-2:, :] = -1
        ###############################################################
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
"""""
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
"""""

'""\n## use those samples of code:\n#In train explore:\nstate, reward, game_over = env.act(action, train=True)\n\n## In Environment exploring:\n# You will have to change n_state to 3 because you will use one more layer!\nreward = 0\nif train:\n    reward = -self.malus_position[self.x, self.y]\nself.malus_position[self.x, self.y] = 0.1\n\nreward = reward + self.board[self.x, self.y]\n# 3 "feature" states instead of 2\nstate = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),\n                                self.board.reshape(self.grid_size, self.grid_size,1),\n                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)\n'

In [19]:
# Training
size = 13
T = 200
temperature = 0.3
epochs_train = 51
epochs_test = 101

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 4, 4, 32)          416       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 32)          4128      
_________________________________________________________________
flatten_5 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                9248      
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 132       
Total params: 13,924
Trainable params: 13,924
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/051 | Epsilon 0.5400 | Loss 3.6607 | Win/lose count 8.5/22.10000000000004 (-13.60000000000004)
Epoch 001/051 | Epsilon 0.48

In [20]:
# Evaluation
size = 13
T = 200
temperature = 0.3
epochs_train = 51
epochs_test = 101

test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Test 0: Win/lose count 6.0/0. Average score (6.0)
Test 1: Win/lose count 18.0/1.0. Average score (11.5)
Test 2: Win/lose count 16.0/2.0. Average score (12.333333333333334)
Test 3: Win/lose count 24.5/2.0. Average score (14.875)
Test 4: Win/lose count 13.5/3.0. Average score (14.0)
Test 5: Win/lose count 16.5/2.0. Average score (14.083333333333334)
Test 6: Win/lose count 17.0/2.0. Average score (14.214285714285714)
Test 7: Win/lose count 13.0/4.0. Average score (13.5625)
Test 8: Win/lose count 10.5/1.0. Average score (13.11111111111111)
Test 9: Win/lose count 26.5/5.0. Average score (13.95)
Test 10: Win/lose count 19.5/1.0. Average score (14.363636363636363)
Test 11: Win/lose count 21.0/2.0. Average score (14.75)
Test 12: Win/lose count 8.0/0. Average score (14.23076923076923)
Test 13: Win/lose count 20.0/2.0. Average score (14.5)
Test 14: Win/lose count 24.0/2.0. Average score (15.0)
Test 15: Win/lose count 15.0/0. Average score (15.0)
Test 16: Win/lose count 13.5/0. Average score (14.

The results are a lot better with an encouraged exploration. The score for the CNN increases from $5.5$ to $18$ for a temperature of $0.3$.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [21]:
class DQN_Mimick(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=3):
        super(DQN_Mimick, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        
        for i in range(self.batch_size):
            # We draw a realisation from the pool in order to learn from it
            s_batch, a_batch = self.memory.random_access()
            #We add the drawn input state to the training batch
            input_states[i, :, :, :] = s_batch
            # We add the action chosen in the memory
            reward_batch = np.zeros(4)
            reward_batch[a_batch] = 10
            target_q[i,:] = reward_batch

        
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

        
class DQN_Mimick_CNN(DQN_Mimick):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_Mimick_CNN, self).__init__(*args,**kwargs)

        model = Sequential()
        model.add(Conv2D(64,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Flatten())
        model.add(Dense(64, activation ='relu'))
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        print(self.model.summary())
        
    def learned_act(self, s):
        scores = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        action = np.argmax(scores)
        return action

In [22]:
def train_Mimick(expert, agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        loss = 0
        
        while not game_over:
            # The expert performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy to the learning agent
            loss += agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [23]:
# Training
size = 13
T = 200
temperature = 0.3
epochs_train = 51
epochs_test = 101

expert = agent

agent_n = DQN_Mimick_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_Mimick(expert,agent_n, env, epochs_train, prefix='cnn_train_Mimick')
HTML(display_videos('cnn_train_Mimick50.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 4, 4, 64)          832       
_________________________________________________________________
flatten_6 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                65600     
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 260       
Total params: 66,692
Trainable params: 66,692
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/051 | Loss 717.4209 | Win/lose count 24.0/1.0 (23.0)
Epoch 001/051 | Loss 107.1380 | Win/lose count 2.5/0 (2.5)
Epoch 002/051 | Loss 239.6606 | Win/lose count 15.5/2.0 (13.5)
Epoch 003/051 | Loss 340.8056 | Win/lose count 24.5/5.0 (19.5)
Epoch 004/051 | L

In [24]:
# Test
size = 13
T = 200
temperature = 0.3
epochs_train = 51
epochs_test = 101

expert = agent

env_test = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test(agent_n,env_test,epochs_test,prefix='cnn_test_mimick')
HTML(display_videos('cnn_test_Mimick100.mp4'))

Test 0: Win/lose count 7.0/2.0. Average score (5.0)
Test 1: Win/lose count 9.5/3.0. Average score (5.75)
Test 2: Win/lose count 9.5/1.0. Average score (6.666666666666667)
Test 3: Win/lose count 15.0/0. Average score (8.75)
Test 4: Win/lose count 0/0. Average score (7.0)
Test 5: Win/lose count 7.0/0. Average score (7.0)
Test 6: Win/lose count 3.0/0. Average score (6.428571428571429)
Test 7: Win/lose count 22.5/1.0. Average score (8.3125)
Test 8: Win/lose count 12.0/0. Average score (8.722222222222221)
Test 9: Win/lose count 15.5/2.0. Average score (9.2)
Test 10: Win/lose count 24.0/3.0. Average score (10.272727272727273)
Test 11: Win/lose count 17.5/2.0. Average score (10.708333333333334)
Test 12: Win/lose count 9.0/0. Average score (10.576923076923077)
Test 13: Win/lose count 14.5/2.0. Average score (10.714285714285714)
Test 14: Win/lose count 4.0/1.0. Average score (10.2)
Test 15: Win/lose count 10.5/0. Average score (10.21875)
Test 16: Win/lose count 13.5/0. Average score (10.4117647

The final average score for the mimicking agent is around $11$, which is better than the first CNN, but lesser than the CNN with an encouraged exploration.  
Actually, the mimicking agent do not try to improve the score but only try to copy the experts agent. It does not try to do better, hence its inability to exceed its score. 

***